# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

# import general stuff
import gym
import random
import torch
import numpy as np
from collections import deque
from scipy import signal
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
# widget bar to display progress
!pip install progressbar
import progressbar as pb

# import specific / own stuff
from ddpg_agent import Agent

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
env = UnityEnvironment(file_name='.\Reacher_Windows_x86_64\Reacher.exe')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]

brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# random actions
actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
print('Random set of actions: {}'.format(actions))

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Number of actions: 4
Random set of actions: [[ 0.88241425  0.37646873 -0.33393117  1.        ]]
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726671e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. [optional] Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

In [5]:
if False:
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    nsteps = 0
    #for _ in range(1, 50):
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        nsteps +=1
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))
    print('Number of time steps is {}'.format(nsteps))

### 4. Let's start training!

When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [6]:
env_info = env.reset(train_mode=True)[brain_name]

#### define DDPG

In [7]:
def ddpg(n_episodes=100, max_t=100, beta_start=1.0, beta_end=0.01, beta_episodeEnd=80):
    """Deep Deterministic Policy Gradient.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps
        beta_start (float): starting value of epsilon, for epsilon-greedy action selection
        beta_end (float): minimum value of epsilon
        beta_episodeEnd (float): number of episodes for which eps_end is reached
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    last_reward_vec = np.zeros(max_t)  # reset reward vector, only for last episode
    beta = beta_start                  # initialize beta, that is factor for adding noise
    beta_decay_linear = (beta_start-beta_end)/beta_episodeEnd # compute linear beta decay rate
    
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]  # reset unity enviroment
        state = env_info.vector_observations[0]            # get the current state 
        score = 0
        for t in range(max_t):
            # choose action and perform it
            actions = agent.act(state, beta)
            env_info = env.step(actions)[brain_name]       # send the actions to the environment
            
            # send env data to agent and initiate learning
            next_state = env_info.vector_observations[0]   # get the next state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0]                  # see if episode has finished
            agent.step(state, actions, reward, next_state, done)
            
            # do some tracking on the rewards
            state = next_state
            score += reward
            if i_episode == n_episodes: last_reward_vec[t] = reward
            if done:
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        beta = max(beta_end, beta_start - beta_decay_linear*i_episode) # decrease beta
        
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 50 == 0:
            # update progress widget bar
            timer.update(i_episode)
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=30:
            # yeaaah, we did it!
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
            break
    # finish timer
    timer.finish()    
    return scores, last_reward_vec

#### create the agent and do the training

In [ ]:
# parameters for learning
n_episodes=4000
max_t=4000
beta_start=1.5
beta_end=0.01
beta_episodeEnd=4000

# create agent and train
agent = Agent(state_size, action_size, 1)        # initialize agent

# uncomment following lines to proceed from save
#checkpoint = torch.load('./data/checkpoint_qnetwork_local_epsEnd1800_1.pth')
#agent.qnetwork_local.load_state_dict(checkpoint['state_dict'])
#checkpoint = torch.load('./data/checkpoint_qnetwork_target_epsEnd1800_1.pth')
#agent.qnetwork_target.load_state_dict(checkpoint['state_dict'])
#checkpoint = torch.load('./data/checkpoint_munetwork_local_epsEnd1800_1.pth')
#agent.munetwork_local.load_state_dict(checkpoint['state_dict'])
#checkpoint = torch.load('./data/checkpoint_munetwork_target_epsEnd1800_1.pth')
#5949agent.munetwork_target.load_state_dict(checkpoint['state_dict'])

# create progress bar to keep track 
widget = ['training loop: ', pb.Percentage(), ' ', 
          pb.Bar(), ' ', pb.ETA() ]
timer = pb.ProgressBar(widgets=widget, maxval=n_episodes).start()

# do actual learning
scores, last_reward_vec = ddpg(n_episodes, max_t, beta_start, beta_end, beta_episodeEnd)


Episode 49	Average Score: 1.08

Episode 50	Average Score: 1.07
Episode 99	Average Score: 2.27

Episode 100	Average Score: 2.26
Episode 149	Average Score: 3.35

Episode 150	Average Score: 3.41
Episode 199	Average Score: 3.83

Episode 200	Average Score: 3.83
Episode 249	Average Score: 4.15

...

Episode 2000	Average Score: 14.89
Episode 2049	Average Score: 15.43

Episode 2050	Average Score: 15.45
Episode 2099	Average Score: 16.29

Episode 2100	Average Score: 16.32
Episode 2149	Average Score: 16.40

Episode 2150	Average Score: 16.38
Episode 2199	Average Score: 21.92

Episode 2200	Average Score: 22.08
Episode 2233	Average Score: 27.51

#### draw the resulting scores

In [ ]:
# butterworth filter with [order] and [cutofffreq/nyquistfreq]
b, a = signal.butter(4, 0.01/0.5)
scores_filtered = signal.filtfilt(b, a, scores)
plt.plot(scores, label='scores')
plt.plot(scores_filtered, label='mean scores filtered')
plt.legend(loc='best')
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.savefig('./data/DDPG_tmax{}_episodes{}_epsEnd{}_twoHiddenlayer.png'.format(max_t, n_episodes, beta_episodeEnd))
plt.show()

In [ ]:
# save data
with open('./data/scores_DDPG_tmax{}_episodes{}_epsEnd{}.pkl'.format(max_t, n_episodes, beta_episodeEnd), 'wb') as f:
    pickle.dump(scores, f)

In [ ]:
#with open('./data/scores_DDPG_tmax2000_episodes500.pkl', 'rb') as f:
#    scores = pickle.load(f)

In [ ]:
# save learned agent if desired
checkpoint = {'input_size': state_size+action_size,
              'output_size': 1,
              'hidden_layers': [each.out_features for each in agent.qnetwork_local.hidden_layers],
              'state_dict': agent.qnetwork_local.state_dict()}
torch.save(checkpoint, './data/checkpoint_qnetwork_local.pth')

checkpoint = {'input_size': state_size+action_size,
              'output_size': 1,
              'hidden_layers': [each.out_features for each in agent.qnetwork_local.hidden_layers],
              'state_dict': agent.qnetwork_local.state_dict()}
torch.save(checkpoint, './data/checkpoint_qnetwork_target.pth')

checkpoint = {'input_size': state_size,
              'output_size': action_size,
              'hidden_layers': [each.out_features for each in agent.munetwork_local.hidden_layers],
              'state_dict': agent.munetwork_local.state_dict()}
torch.save(checkpoint, './data/checkpoint_munetwork_local.pth')

checkpoint = {'input_size': state_size,
              'output_size': action_size,
              'hidden_layers': [each.out_features for each in agent.munetwork_target.hidden_layers],
              'state_dict': agent.munetwork_target.state_dict()}
torch.save(checkpoint, './data/checkpoint_munetwork_target.pth')


### 5. watch trained agent playing

In [ ]:
# load from save
agent = Agent(state_size, action_size, 1)        # initialize agent
checkpoint = torch.load('./data/checkpoint_qnetwork_local_epsEnd1800_1.pth')
agent.qnetwork_local.load_state_dict(checkpoint['state_dict'])
checkpoint = torch.load('./data/checkpoint_qnetwork_target_epsEnd1800_1.pth')
agent.qnetwork_target.load_state_dict(checkpoint['state_dict'])
checkpoint = torch.load('./data/checkpoint_munetwork_local_epsEnd1800_1.pth')
agent.munetwork_local.load_state_dict(checkpoint['state_dict'])
checkpoint = torch.load('./data/checkpoint_munetwork_target_epsEnd1800_1.pth')
agent.munetwork_target.load_state_dict(checkpoint['state_dict'])

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
all_actions = np.zeros((num_agents, action_size))        # initialize the actions (for each agent)
all_scores = np.zeros(num_agents) 
for _ in range(1, 500):
    actions = agent.act(states, beta = 0., add_noise = False)      # select actions without 
    all_actions = np.append(all_actions, actions, axis=0)
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    all_scores = np.append(all_scores, env_info.rewards)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

In [ ]:
for i in range(4):
    plt.plot(all_actions[:,i])
plt.show()


Close the environment.

In [ ]:
env.close()